In [11]:
# https://qiskit.org/documentation/tutorials/circuits/1_getting_started_with_qiskit.html
# https://qiskit.org/documentation/machine-learning/tutorials/03_quantum_kernel.html
# https://github.com/marcinjastrzebski8/QuantumKernelEstimation  (useful as a small guide)
# https://xavierbourretsicotte.github.io/SVM_implementation.html
# https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf  ===> how to use cvxopt

# TODO : see if one can improve accuracy
# TODO : graphics to draw the separation hyperplane and see if it has the margins promised in the paper
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, BasicAer, transpile, Aer, IBMQ
from qiskit.circuit import ParameterVector, Parameter
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel
from cvxopt import matrix, solvers
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame, read_csv
import os
from math import pi

seed = 12345
algorithm_globals.random_seed = seed
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams['font.size'] = 12

In [26]:
"""
14,-1
15,-1
16,1
19,1
20,-1
22,1
"""
X = [14, 16, 20, 22]
num_samples_train = 2
num_samples_test = 2
thetas = []
# k is 1 here
for idx in range(len(X)):
    x = X[idx]
    val_1 = format(x, '07b')
    val_2 = format(5*x, '07b')
    thetas.append([pi * int(x) for x in list(val_1)])
    print(x, val_1, val_2)
    
features_train = thetas[0:2]
features_test = thetas[2:]

labels = [-1, 1, -1, 1]
labels_train = [-1, 1]
labels_test = [-1, 1]
print(thetas)

14 0001110 1000110
16 0010000 1010000
20 0010100 1100100
22 0010110 1101110
[[0.0, 0.0, 0.0, 3.141592653589793, 3.141592653589793, 3.141592653589793, 0.0], [0.0, 0.0, 3.141592653589793, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.141592653589793, 0.0, 3.141592653589793, 0.0, 0.0], [0.0, 0.0, 3.141592653589793, 0.0, 3.141592653589793, 3.141592653589793, 0.0]]


In [48]:
num_qubits = 7
parameters =[Parameter('θ_' + str(i)) for i in range(num_qubits)]
qc = QuantumCircuit(num_qubits, num_qubits)

for idx in range(len(parameters)):
    qc.rx(parameters[idx], idx)
    qc.rz(parameters[idx], idx)


qc.h(0)
qc.toffoli(2, 5, 1)

qc.x(3)
qc.toffoli(3,5,2)
qc.x(3)

qc.toffoli(0,4,3)

qc.cx(6, 4)
qc.cx(6, 5)

qc.x(4)
qc.toffoli(4,5,6)
qc.x(4)
print(qc.draw())

for idx in range(num_qubits):
    qc.measure(idx, idx)

    
binding_dict = {}
for i in range(num_qubits):
    binding_dict[parameters[i]] = thetas[3][i]
    
print(binding_dict)

circuit_att = qc.bind_parameters(binding_dict)

print(circuit_att)

backend = BasicAer.get_backend('qasm_simulator')
job = backend.run(transpile(circuit_att, backend))
counts = job.result().get_counts()
print(counts)

     ┌─────────┐┌─────────┐┌───┐                                        
q_0: ┤ Rx(θ_0) ├┤ Rz(θ_0) ├┤ H ├─────────────────■──────────────────────
     ├─────────┤├─────────┤├───┤                 │                      
q_1: ┤ Rx(θ_1) ├┤ Rz(θ_1) ├┤ X ├─────────────────┼──────────────────────
     ├─────────┤├─────────┤└─┬─┘     ┌───┐       │                      
q_2: ┤ Rx(θ_2) ├┤ Rz(θ_2) ├──■───────┤ X ├───────┼──────────────────────
     ├─────────┤├─────────┤  │  ┌───┐└─┬─┘┌───┐┌─┴─┐                    
q_3: ┤ Rx(θ_3) ├┤ Rz(θ_3) ├──┼──┤ X ├──■──┤ X ├┤ X ├────────────────────
     ├─────────┤├─────────┤  │  └───┘  │  └───┘└─┬─┘┌───┐┌───┐     ┌───┐
q_4: ┤ Rx(θ_4) ├┤ Rz(θ_4) ├──┼─────────┼─────────■──┤ X ├┤ X ├──■──┤ X ├
     ├─────────┤├─────────┤  │         │            └─┬─┘├───┤  │  └───┘
q_5: ┤ Rx(θ_5) ├┤ Rz(θ_5) ├──■─────────■──────────────┼──┤ X ├──■───────
     ├─────────┤├─────────┤                           │  └─┬─┘┌─┴─┐     
q_6: ┤ Rx(θ_6) ├┤ Rz(θ_6) ├────────────────────────

In [33]:
### Custom Feature Map

# for now, we have as many qubits as features

      
# creating the feature map and extracting the kernel matrix
backend = QuantumInstance(Aer.get_backend("qasm_simulator"), shots=1024, seed_simulator=seed, seed_transpiler=seed)


foldername = "./all_results_for_DLP_rigsup_ds/"
os.mkdir(foldername)
qk = QuantumKernel(feature_map=qc, quantum_instance=backend)
K = qk.evaluate(features_train)

numerical_results_filename = foldername + 'accuracy_results.txt'
num_features = 7
lambda_best_acc = -1
best_acc = -1
# Computing the alphas of the SVM algorithm
for lambda_svm in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    Q = np.zeros((num_samples_train, num_samples_train))
    for i in range(num_samples_train):
        for j in range(num_samples_train):
            Q[i][j] = labels[i] * labels[j] * K[i][j]
    Pm = np.eye(num_samples_train) / lambda_svm + Q

    P = matrix(Pm, tc='d')
    q = matrix(-np.ones(num_samples_train), tc='d')
    G = matrix(-np.eye(num_samples_train), tc='d')
    h = matrix(np.zeros(num_samples_train), tc='d')

    sol = solvers.qp(P,q,G,h)
    alpha = (np.array(sol['x']).T)[0]
    good = 0
    bad = 0
    for test_idx in range(num_samples_test):
        new_sum = 0

        for train_idx in range(num_samples_train):
            pair = np.zeros((2, num_features))
            pair[0] = features_train[train_idx]
            pair[1] = features_test[test_idx]
            current_K = qk.evaluate(pair)
            new_sum = new_sum + alpha[train_idx] * labels_train[train_idx] * current_K[0][1] # x_i vs x (test)

        if new_sum * labels_test[test_idx] >= 0: # different classes = different sign = negative product
            good = good + 1
        #    print("Good for " + str(test_idx))
        else:
            bad = bad + 1
        #    print("Bad for " + str(test_idx))

        if round(good / (good + bad), 4) * 100 > best_acc:
            best_acc = round(good / (good + bad), 4) * 100
            lambda_best_acc = lambda_svm

    with open(numerical_results_filename, 'a') as f:
        print("Lambda is: ", lambda_svm, file=f)
        print("Well classifed: ", good, file=f)
        print("Bad classified: ", bad, file=f)
        print("Accuracy: ", round(good / (good + bad), 4) * 100, file=f)
        print('\n################################\n', file=f)

    """
    # Plot for best lambda
    lambda_svm = lambda_best_acc # we plot for the best lambda
    Q = np.zeros((num_samples_train, num_samples_train))
    for i in range(num_samples_train):
        for j in range(num_samples_train):
            Q[i][j] = labels[i] * labels[j] * K[i][j]
    Pm = np.eye(num_samples_train) / lambda_svm + Q

    P = matrix(Pm, tc='d')
    q = matrix(-np.ones(num_samples_train), tc='d')
    G = matrix(-np.eye(num_samples_train), tc='d')
    h = matrix(np.zeros(num_samples_train), tc='d')

    sol = solvers.qp(P,q,G,h)
    alpha = (np.array(sol['x']).T)[0]
    print(alpha[0:4])
    good = 0
    bad = 0

    w_0 = 0
    w_1 = 0
    w_2 = 0
    w_3 = 0

    for test_idx in range(num_samples_test):
        new_sum = 0

        for train_idx in range(num_samples_train):
            pair = np.zeros((2, num_features))
            pair[0] = features_train[train_idx]
            pair[1] = features_test[test_idx]
            current_K = qk.evaluate(pair)
            new_sum = new_sum + alpha[train_idx] * labels_train[train_idx] * current_K[0][1] # x_i vs x (test)
            w_0 = w_0 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][0]
            w_1 = w_1 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][1]
            w_2 = w_2 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][2]
            w_3 = w_3 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][3]

    ##

    fig, ax = plt.subplots()

    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='feature 1', y='feature 2', label=key, color=colors[key])

    x0_sample = np.linspace(-10.0, 10.0, num=1000)
    x1_sample = -w_0 * x0_sample / w_1
    ax.plot(x0_sample, x1_sample)

    plt.savefig(foldername + 'pauli_fm_hyperplane_f1_f2.png')
    plt.show()

    ##

    fig, ax = plt.subplots()

    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='feature 2', y='feature 3', label=key, color=colors[key])

    x0_sample = np.linspace(-10.0, 10.0, num=1000)
    x1_sample = -w_1 * x0_sample / w_2
    ax.plot(x0_sample, x1_sample)

    plt.savefig(foldername + 'pauli_fm_hyperplane_f2_f3.png')
    plt.show()

    ##

    fig, ax = plt.subplots()

    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='feature 3', y='feature 4', label=key, color=colors[key])

    x0_sample = np.linspace(-10.0, 10.0, num=1000)
    x1_sample = -w_2 * x0_sample / w_3
    ax.plot(x0_sample, x1_sample)

    plt.savefig(foldername + 'pauli_fm_hyperplane_f3_f4.png')
    plt.show()
    """

     pcost       dcost       gap    pres   dres
 0: -9.9900e-04 -2.9950e-03  2e-03  0e+00  1e+00
 1: -9.9900e-04 -1.0190e-03  2e-05  3e-19  1e-02
 2: -9.9900e-04 -9.9920e-04  2e-07  3e-19  1e-04
 3: -9.9900e-04 -9.9900e-04  2e-09  0e+00  1e-06
 4: -9.9900e-04 -9.9900e-04  2e-11  3e-19  1e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9000e-03 -2.9508e-02  2e-02  0e+00  1e+00
 1: -9.9009e-03 -1.0098e-02  2e-04  2e-18  1e-02
 2: -9.9010e-03 -9.9030e-03  2e-06  7e-18  1e-04
 3: -9.9010e-03 -9.9010e-03  2e-08  2e-18  1e-06
 4: -9.9010e-03 -9.9010e-03  2e-10  0e+00  1e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.0278e-02 -2.5694e-01  2e-01  0e+00  1e+00
 1: -9.0858e-02 -9.2633e-02  2e-03  4e-17  3e-02
 2: -9.0909e-02 -9.0927e-02  2e-05  2e-17  3e-04
 3: -9.0909e-02 -9.0909e-02  2e-07  2e-17  3e-06
 4: -9.0909e-02 -9.0909e-02  2e-09  0e+00  3e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 

In [ ]:
numerical_results_filename = foldername + 'accuracy_results.txt'

lambda_best_acc = -1
best_acc = -1
# Computing the alphas of the SVM algorithm
for lambda_svm in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    Q = np.zeros((num_samples_train, num_samples_train))
    for i in range(num_samples_train):
        for j in range(num_samples_train):
            Q[i][j] = labels[i] * labels[j] * K[i][j]
    Pm = np.eye(num_samples_train) / lambda_svm + Q

    P = matrix(Pm, tc='d')
    q = matrix(-np.ones(num_samples_train), tc='d')
    G = matrix(-np.eye(num_samples_train), tc='d')
    h = matrix(np.zeros(num_samples_train), tc='d')

    sol = solvers.qp(P,q,G,h)
    alpha = (np.array(sol['x']).T)[0]
    good = 0
    bad = 0
    for test_idx in range(num_samples_test):
        new_sum = 0

        for train_idx in range(num_samples_train):
            pair = np.zeros((2, num_features))
            pair[0] = features_train[train_idx]
            pair[1] = features_test[test_idx]
            current_K = qk.evaluate(pair)
            new_sum = new_sum + alpha[train_idx] * labels_train[train_idx] * current_K[0][1] # x_i vs x (test)

        if int(new_sum/abs(new_sum)) == int(labels_test[test_idx]):
            good = good + 1
        #    print("Good for " + str(test_idx))
        else:
            bad = bad + 1
        #    print("Bad for " + str(test_idx))
        
        if round(good / (good + bad), 4) * 100 > best_acc:
            best_acc = round(good / (good + bad), 4) * 100
            lambda_best_acc = lambda_svm
    
    with open(numerical_results_filename, 'a') as f:
        print("Lambda is: ", lambda_svm, file=f)
        print("Well classifed: ", good, file=f)
        print("Bad classified: ", bad, file=f)
        print("Accuracy: ", round(good / (good + bad), 4) * 100, file=f)
        print('\n################################\n', file=f)
        
        
    

In [ ]:
# Plot for best lambda
lambda_svm = lambda_best_acc # we plot for the best lambda
Q = np.zeros((num_samples_train, num_samples_train))
for i in range(num_samples_train):
    for j in range(num_samples_train):
        Q[i][j] = labels[i] * labels[j] * K[i][j]
Pm = np.eye(num_samples_train) / lambda_svm + Q

P = matrix(Pm, tc='d')
q = matrix(-np.ones(num_samples_train), tc='d')
G = matrix(-np.eye(num_samples_train), tc='d')
h = matrix(np.zeros(num_samples_train), tc='d')

sol = solvers.qp(P,q,G,h)
alpha = (np.array(sol['x']).T)[0]
print(alpha[0:4])
good = 0
bad = 0

w_0 = 0
w_1 = 0
w_2 = 0
w_3 = 0

for test_idx in range(num_samples_test):
    new_sum = 0

    for train_idx in range(num_samples_train):
        pair = np.zeros((2, num_features))
        pair[0] = features_train[train_idx]
        pair[1] = features_test[test_idx]
        current_K = qk.evaluate(pair)
        new_sum = new_sum + alpha[train_idx] * labels_train[train_idx] * current_K[0][1] # x_i vs x (test)
        w_0 = w_0 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][0]
        w_1 = w_1 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][1]
        w_2 = w_2 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][2]
        w_3 = w_3 + alpha[train_idx] * labels_train[train_idx] * features_train[train_idx][3]

##

fig, ax = plt.subplots()

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='feature 1', y='feature 2', label=key, color=colors[key])
    
x0_sample = np.linspace(-10.0, 10.0, num=1000)
x1_sample = -w_0 * x0_sample / w_1
ax.plot(x0_sample, x1_sample)

plt.savefig(foldername + 'pauli_fm_hyperplane_f1_f2.png')
plt.show()

##

fig, ax = plt.subplots()

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='feature 2', y='feature 3', label=key, color=colors[key])

x0_sample = np.linspace(-10.0, 10.0, num=1000)
x1_sample = -w_1 * x0_sample / w_2
ax.plot(x0_sample, x1_sample)

plt.savefig(foldername + 'pauli_fm_hyperplane_f2_f3.png')
plt.show()

##

fig, ax = plt.subplots()

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='feature 3', y='feature 4', label=key, color=colors[key])

x0_sample = np.linspace(-10.0, 10.0, num=1000)
x1_sample = -w_2 * x0_sample / w_3
ax.plot(x0_sample, x1_sample)

plt.savefig(foldername + 'pauli_fm_hyperplane_f3_f4.png')
plt.show()